In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from math import ceil

data = pd.read_excel('data.xlsx', header=None).values.flatten()

In [24]:
requirements = data
requirements

array([ 11,   5,   4,   7,  16,   6,   5,   7,  13,   6,   5,   7,  12,
         5,   4,   6,   9,   5,   5,  11,  29,  21,  17,  20,  27,  13,
         9,  10,  16,   6,   5,   7,  11,   5,   5,   6,  12,   7,   7,
        10,  15,  10,   9,  11,  15,  10,  10,  16,  26,  21,  23,  36,
        50,  45,  45,  49,  57,  43,  40,  44,  52,  43,  42,  45,  52,
        41,  39,  41,  48,  35,  34,  35,  42,  34,  36,  43,  55,  48,
        54,  65,  80,  70,  74,  85, 101,  89,  88,  90, 100,  87,  88,
        89, 104,  89,  89,  90, 106,  96,  94,  99, 109,  99,  96, 102],
      dtype=int64)

In [25]:
total_week = 8
requirements = requirements[0:total_week]
K = 20
waste_rate = 0.1

In [26]:
working_hands = cp.Parameter(total_week)
waste_hands = cp.Parameter(total_week)
working_hands.value = 4 * requirements
waste_hands.value = np.array([
    round(4 * x * waste_rate) if x % 1 != 0.5 else ceil(4 * x * waste_rate) 
    for x in requirements
])


teaching_hands = cp.Variable(total_week, integer=True)
resting_hands = cp.Variable(total_week, integer=True)
new_hands = cp.Variable(total_week, integer=True)

# 机械手购买的分段点
hand_bp = cp.Parameter((1, 4))
hand_bp.value = np.array([0, 20, 40, 200]).reshape(1, 4)
hand_bp_values = cp.Parameter((1, 4))
hand_bp_values.value = np.array([0, 20 * 100, 20 * 100 + 20 * 90, 20 * 100 + 20 * 90 + 160 * 80]).reshape(1, 4)

hand_z = cp.Variable((total_week, 4))
hand_y = cp.Variable((total_week, 3), integer=True)

# 容器艇购买的分段点
body_bp = cp.Parameter((1, 4))
body_bp.value = np.array([0, 5, 10, 100]).reshape(1, 4)
body_bp_values = cp.Parameter(4)
body_bp_values.value = np.array([0, 5 * 200, 5 * 200 + 5 * 180, 5 * 200 + 5 * 180 + 90 * 160])

body_z = cp.Variable((total_week, 4))
body_y = cp.Variable((total_week, 4), integer=True)

In [27]:
working_hands.value, waste_hands.value

(array([44, 20, 16, 28, 64, 24, 20, 28], dtype=int64),
 array([4, 2, 2, 3, 6, 2, 2, 3]))

In [28]:
cons = [
    resting_hands[0] + teaching_hands[0] + working_hands[0] == 50,
    K * teaching_hands >= new_hands,

    teaching_hands >= 0,
    resting_hands >= 0,
    new_hands >= 0,

    # 分段函数引入的约束
    hand_z[:, 0] <= hand_y[:, 0],
    hand_z[:, 1] <= hand_y[:, 0] + hand_y[:, 1],
    hand_z[:, 2] <= hand_y[:, 1] + hand_y[:, 2],
    hand_z[:, 3] <= hand_y[:, 2],
    cp.sum(hand_z, axis=1) == 1,
    hand_z >= 0,
    cp.sum(hand_y, axis=1) == 1,
    hand_y >= 0,
    hand_y <= 1,

    new_hands == cp.sum(cp.multiply(hand_z, hand_bp))
]

In [29]:
for t in range(total_week - 1):
    cons.append(resting_hands[t + 1] >= working_hands[t] - waste_hands[t]),
    cons.append(resting_hands[t + 1] + working_hands[t + 1] + teaching_hands[t + 1] == \
        resting_hands[t] + working_hands[t] + teaching_hands[t] + new_hands[t] - waste_hands[t]
        )

In [30]:
obj = cp.Minimize(
    cp.sum(cp.multiply(hand_z, hand_bp_values))\
    + 5 * cp.sum(resting_hands)\
    + 10 * cp.sum(teaching_hands + new_hands)
)

In [31]:
prob = cp.Problem(obj, cons)
prob.solve(verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 03 12:39:35 AM: Your problem has 80 variables, 29 constraints, and 24 parameters.
(CVXPY) May 03 12:39:35 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 03 12:39:35 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 03 12:39:35 AM: Compiling problem (target solver=CPLEX).
(CVXPY) May 03 12:39:35 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> CPLEX
(CVXPY) May 03 12:39:35 AM: Applying reduction Dcp2Cone
(CVXPY) May 03 12:39:35 AM: Applying reduction CvxAttr2Constr
(CVX

5190.0

In [32]:
working_hands.value, resting_hands.value

(array([44, 20, 16, 28, 64, 24, 20, 28], dtype=int64),
 array([  5.,  40.,  57.,  58.,  34.,  83., 100., 105.]))

In [33]:
hand_z.value

array([[ 1.  , -0.  ,  0.  ,  0.  ],
       [ 1.  , -0.  ,  0.  ,  0.  ],
       [ 1.  , -0.  ,  0.  ,  0.  ],
       [ 1.  , -0.  ,  0.  ,  0.  ],
       [ 1.  , -0.  ,  0.  ,  0.  ],
       [ 1.  , -0.  ,  0.  ,  0.  ],
       [ 1.  , -0.  ,  0.  ,  0.  ],
       [ 0.25,  0.75,  0.  ,  0.  ]])

In [34]:
new_hands.value, teaching_hands.value

(array([15., 15., 15., 15., 15., 15., 15., 15.]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]))